In [1]:
import tensorflow as tf
from keras.layers import Input, Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D, concatenate, Activation
from keras.applications.mobilenet import MobileNet as Net

from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.layers import Input, merge
from keras.optimizers import Adam
import cv2
import numpy as np
import os
#from tqdm import tqdm
from keras.models import Sequential

from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import time
import glob

ALPHA = 0.7 # Triplet Loss Parameter

from keras.layers import Input,Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D,concatenate,Activation
from keras.applications.xception import Xception as Net
from keras.preprocessing import image
from keras.applications.xception import preprocess_input
from keras.models import Model

import matplotlib.pyplot as plt

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix
import random as rn

np.random.seed(0)
tf.set_random_seed(3)

from sklearn.model_selection import cross_val_score

/Applications/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# run the script containing all framework related functions
%run fn_utils.py

In [3]:
# load the CNN model
pretrained_model= keras.applications.mobilenet.MobileNet()
layers_needed=pretrained_model.layers[0:82]
#Defining the Sequential model
baseline_model=Sequential()
for layer in layers_needed:
    baseline_model.add(layer)
    
layers_to_not_train= baseline_model.layers[0:79]

for layer in layers_to_not_train:
    layer.trainable=False
    
triplet_model=create_model_new(128, 128, 3, baseline_model)
triplet_model.load_weights('/Users/ochhab3/Documents/ojaswinich_github/signature_verification/model_weights/triplet_weights_new_e3.h5')

/Users/ochhab3/Documents/ojaswinich_github/signature_verification/fn_utils.py:71: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  loss = merge([anchor, positive, negative], mode=triplet_loss, output_shape=(1,))
/Applications/anaconda/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor (InputLayer)             (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
positive (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
negative (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       multiple             3228864     anchor[0][0]                     
                                                                 positive[0][0]                   
          

In [4]:
#Find the embeddings from the cnn_model
cnn_model = Model(inputs=triplet_model.input,
                                 outputs=triplet_model.get_layer('global_max_pooling2d_1').output)

In [5]:
# load the logistic model
import pickle
log_model_loaction='/Users/ochhab3/Documents/ojaswinich_github/signature_verification/model_weights/log_model_e3.pkl'
log_model_saved=open(log_model_loaction, 'rb')
log_model=pickle.load(log_model_saved)

In [6]:
# creating function to input images from folder 1 and folder 2 and save embeddings in database 
def prepare_database_embed(location, cnn_model):
    database = {}

    # load all the images of individuals to recognize into the database
    for file in glob.glob(location):
        identity = os.path.splitext(os.path.basename(file))[0]
        #print (identity)
        identity_array = img_path_to_array(file)
        identity_embed=cnn_model.predict([identity_array,identity_array,identity_array])
        database[identity] = identity_embed

    return database

In [9]:
# creating base images database dictionary
folder1_location='/Users/ochhab3/Documents/ojaswinich_github/signature_verification/DataBase/*' #input path here
folder1_database = prepare_database_embed(folder1_location, cnn_model)

In [10]:
folder1_database

{'U11001_152': array([[0., 0., 0., ..., 6., 0., 0.]], dtype=float32),
 'U11001_188': array([[0.       , 3.9395442, 0.       , ..., 1.0244675, 4.193406 ,
         0.       ]], dtype=float32),
 'U11001_79': array([[0.        , 4.196162  , 0.        , ..., 0.61037445, 0.        ,
         0.        ]], dtype=float32),
 'U11002_107': array([[0.        , 6.        , 0.        , ..., 2.912363  , 0.34511495,
         0.        ]], dtype=float32),
 'U11002_132': array([[0.53647804, 2.0188456 , 0.        , ..., 4.700926  , 3.380831  ,
         0.        ]], dtype=float32),
 'U11002_233': array([[0.       , 2.20398  , 0.       , ..., 3.130127 , 2.6565282,
         1.7142801]], dtype=float32),
 'U11003_103': array([[0.       , 0.       , 0.       , ..., 3.8456745, 0.       ,
         0.       ]], dtype=float32),
 'U11003_308': array([[0.       , 3.2204118, 3.5960693, ..., 4.292263 , 0.       ,
         0.       ]], dtype=float32),
 'U11003_9': array([[1.1514797, 0.       , 0.       , ..., 2.23516

In [11]:
# creating final test images database dictionary
folder2_location='/Users/ochhab3/Documents/ojaswinich_github/signature_verification/Test/*' #input path here
folder2_database = prepare_database_embed(folder2_location, cnn_model)

In [12]:
folder2_database

{'U11001_124': array([[6.       , 0.       , 0.       , ..., 3.0220928, 0.       ,
         0.       ]], dtype=float32),
 'U11001_125': array([[2.592184  , 4.034378  , 0.83892727, ..., 5.71484   , 0.        ,
         0.        ]], dtype=float32),
 'U11001_127': array([[0.       , 3.6688914, 0.       , ..., 3.1293716, 0.       ,
         0.       ]], dtype=float32),
 'U11001_162': array([[2.2086911, 0.       , 0.       , ..., 1.898035 , 1.5118281,
         0.       ]], dtype=float32),
 'U11001_273': array([[0.7242098 , 0.67688704, 0.        , ..., 5.6955795 , 0.        ,
         0.        ]], dtype=float32),
 'U11001_297': array([[2.9706554, 0.       , 0.       , ..., 1.6925392, 0.       ,
         0.       ]], dtype=float32),
 'U11001_341': array([[0.        , 3.9571164 , 0.8049717 , ..., 0.93590164, 0.        ,
         0.        ]], dtype=float32),
 'U11001_343': array([[6.        , 2.9423501 , 0.        , ..., 0.31074524, 6.        ,
         0.        ]], dtype=float32),
 'U11001

In [13]:
# empty list for probabilities
final_pred_prob=[['Filename', 'Prob1', 'Prob2', 'Prob3']]

In [14]:
for test_id in folder2_database:
    # assuming test_id starts with a letter. if not remove last part
    person_id= test_id.split('_')[0][1:]
    test_emb=folder2_database[test_id]
    pred_prob_class = [test_id]
    orig_ids=[]
    for i in folder1_database.keys():
          if i.split('_')[0][1:]==person_id:
                orig_ids.append(i)
    for i in orig_ids:
        anc_emb=folder1_database[i]
        diff_emb=anc_emb-test_emb
        pred_prob=log_model.predict_proba(diff_emb)[:,1][0]
        #print (pred_prob)
        pred_prob_class.append(pred_prob)
        #print (pred_prob_class)
    # getting the mode of pred_class
    final_pred_prob.append(pred_prob_class)

In [15]:
final_pred_prob

[['Filename', 'Prob1', 'Prob2', 'Prob3'],
 ['U11007_425',
  0.27728080397675714,
  4.0024272284929624e-07,
  0.3660075346325159],
 ['U11010_471',
  2.6607107959301208e-08,
  0.0205391320480008,
  8.035299203976292e-10],
 ['U11004_46', 0.5247942825520332, 0.0002799394958201041, 0.9200534309932326],
 ['U11040_123',
  4.315383563031306e-18,
  5.803087724528132e-18,
  1.685263771271635e-22],
 ['U11013_388',
  4.859615192420939e-06,
  1.4922412624229086e-06,
  0.5537360213246318],
 ['U11020_53',
  1.711724319132154e-12,
  0.0003562138979494925,
  0.007364577318689973],
 ['U11035_177',
  0.9124731585545257,
  1.9384271713818084e-12,
  6.820557678095813e-07],
 ['U11039_347',
  5.061201265160474e-12,
  4.13817276324186e-19,
  2.2581201346508423e-06],
 ['U11013_148',
  3.5566189112864645e-06,
  1.0921294972011562e-06,
  0.4759254034348799],
 ['U11003_174',
  4.464296256713764e-16,
  6.923568287954518e-08,
  2.8229562079308392e-11],
 ['U11001_273',
  7.057675643866225e-17,
  1.5250948383474613e-

In [16]:
# creating an empty list which will be appended 
final_predictions_any=[['Filename', 'Forged']]

In [17]:
# proabability threshold to take for forged class
k=0.4

In [18]:
# if any one is forged, take forged

for test_id in folder2_database:
    person_id= test_id.split('_')[0][1:]
    test_emb=folder2_database[test_id]
    pred_prob_class = []
    orig_ids=[]
    for i in folder1_database.keys():
          if i.split('_')[0][1:]==person_id:
                orig_ids.append(i)
    for i in orig_ids:
        anc_emb=folder1_database[i]
        diff_emb=anc_emb-test_emb
        pred_prob=log_model.predict_proba(diff_emb)[:,1][0]
        #print (pred_prob)
        pred_prob_class.append(pred_prob)
        #print (pred_prob_class)
    # taking final class as forged if any probability >= k, otherwise genuine
    if (pred_prob_class[0] >= k or pred_prob_class[1] >= k or pred_prob_class[2] >= k):
        output='Yes'
    else:
        output='No'
    output_entry=[test_id, output]    
    
    final_predictions_any.append(output_entry)

In [19]:
# get total number of genuine and forged class
x1=np.array(final_predictions_any)[1:]
x2=x1[:,1]
num_orig=len(x2[x2=='No'])
num_forg=len(x2[x2=='Yes'])
total=num_orig+num_forg
ratio_forg=num_forg/total
print(num_orig,num_forg,ratio_forg)

290 100 0.2564102564102564


In [20]:
# creating an empty list which will be appended 
final_predictions_mode=[['Filename', 'Decision']]

In [21]:
# if mode is forged, take forged

for test_id in folder2_database:
    person_id= test_id.split('_')[0][1:]
    test_emb=folder2_database[test_id]
    pred_prob_class = []
    orig_ids=[]
    for i in folder1_database.keys():
          if i.split('_')[0][1:]==person_id:
                orig_ids.append(i)
    for i in orig_ids:
        anc_emb=folder1_database[i]
        diff_emb=anc_emb-test_emb
        pred_prob=log_model.predict_proba(diff_emb)[:,1][0]
        #print (pred_prob)
        pred_prob_class.append(pred_prob)
        #print (pred_prob_class)
    # taking final class as forged if mode of probability >= k, otherwise genuine
    a=sum(i >= k for i in pred_prob_class)
    if (a>=2):
        output='Yes'
    else:
        output='No'
    output_entry=[test_id, output]    
    
    final_predictions_mode.append(output_entry)

In [22]:
# get total number of genuine and forged class
x1=np.array(final_predictions_mode)[1:]
x2=x1[:,1]
num_orig=len(x2[x2=='No'])
num_forg=len(x2[x2=='Yes'])
total=num_orig+num_forg
ratio_forg=num_forg/total
print(num_orig,num_forg,ratio_forg)

352 38 0.09743589743589744


In [23]:
# converting the final_predictions_any list to .csv file
import csv
final_predictions_path='final_predictions_any.csv' # give path here
myFile = open(final_predictions_path, 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(final_predictions_any)

In [24]:
# converting the final_predictions_mode list to .csv file
import csv
final_predictions_path='final_predictions_mode.csv' # give path here
myFile = open(final_predictions_path, 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(final_predictions_mode)

In [25]:
# converting the final_pred_prob list to .csv file
import csv
final_pred_prob_path='final_probabilities_test.csv' # give path here
myFile = open(final_pred_prob_path, 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(final_pred_prob)